### 1. General info of dataset GSE221776

This is the Jupyter Notebook for dataset GSE221776. Its dataset includes one annotation and uni txt file for both CD4 and CD8 T cells. As seen below, in the txt file, each row is a gene and each column is a cell.

Thus, we need to transform this txt file and generate an overall AnnData object for all samples. 



In [1]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

In [26]:
# inspect the dataset
cd4_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE221776/GSE221776_sc_PBT_CD4_umi.txt.gz'
cd4_input = pd.read_csv(cd4_path, sep='\t', index_col=0) # the first column contains gene names and is the index

print(cd4_input.head()) 
print(cd4_input.shape) # (15095 rows, 15893 columns)

       gene_name  AAACCTGAGAGTACCG.1  AAACCTGAGCAATATG.1  AAACCTGAGCACACAG.1  \
1     AL627309.1                   0                   0                   0   
2     AP006222.2                   0                   0                   0   
3  RP11-206L10.3                   0                   0                   0   
4  RP11-206L10.2                   0                   0                   0   
5  RP11-206L10.9                   0                   0                   0   

   AAACCTGAGTGTCCAT.1  AAACCTGAGTTAAGTG.1  AAACCTGCACGGTAAG.1  \
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   
5                   0                   0                   0   

   AAACCTGCAGCCTTTC.1  AAACCTGGTCCATCCT.1  AAACCTGGTTCTGAAC.1  ...  \
1                   0                   0                 

In [27]:
# inspect the annotation file
cd4_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE221776/GSE221776_sc_PBT_CD4_annotation.txt.gz'
cd4_annotation = pd.read_csv(annotation_path, sep='\t', index_col=0) # the first column contains gene names and is the index

cd4_annotation

,clonotype.tag,nCount_RNA,nFeature_RNA,percent.mt,percent.ribo,percent.hs,Run,Project,Species,donor,...,n_sG,tra.v,tra.j,trb.v,trb.j,cell_classification_deconv,paper_cluster,Cell.Type,library_name,donor_publicationID
cell_name,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAGTACCG-1,clonotype39,3210,1645,1.59,14.17,0.56,NV035,AdUp01,Hu,BT5,...,NaN,NaN,NaN,TRBV4-1,TRBJ1-2,TREG,6,CD4,PBT_batch1_library1_7donors_Gex,PBT_12
AAACCTGAGCAATATG-1,clonotype866,2791,1502,4.87,13.79,0.11,NV035,AdUp01,Hu,BT1,...,NaN,TRAV1-1,TRAJ11,TRBV20-1,TRBJ1-6,TREG,6,CD4,PBT_batch1_library1_7donors_Gex,PBT_01
AAACCTGAGCACACAG-1,NaN,3270,1433,3.06,21.01,0.40,NV035,AdUp01,Hu,BT1,...,NaN,NaN,NaN,NaN,NaN,CTL,2,CD4,PBT_batch1_library1_7donors_Gex,PBT_01
AAACCTGAGTGTCCAT-1,clonotype583,3695,1668,1.62,16.39,0.27,NV035,AdUp01,Hu,BT7,...,NaN,NaN,NaN,TRBV4-1,TRBJ2-1,CTL,1,CD4,PBT_batch1_library1_7donors_Gex,PBT_02
AAACCTGAGTTAAGTG-1,NaN,3327,1469,4.39,25.51,0.27,NV035,AdUp01,Hu,BT5,...,NaN,NaN,NaN,NaN,NaN,TN,3,CD4,PBT_batch1_library1_7donors_Gex,PBT_12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGGAACAGCACCGTC-12,clonotype46460,1893,912,1.74,28.19,1.27,NV099,AdUp07,Hu,BT38,...,NaN,TRAV8-4,TRAJ17,TRBV19,TRBJ1-1,TN,3,CD4,PBT_batch5_library4_6donors_Gex,PBT_15
TTGGCAAGTTGCGCAC-12,clonotype46539,2922,1179,3.76,31.79,0.31,NV099,AdUp07,Hu,BT46,...,NaN,TRAV13-1,TRAJ49,TRBV20-1,TRBJ2-3,TCM,0,CD4,PBT_batch5_library4_6donors_Gex,PBT_34
TTGTAGGGTAGCGCTC-12,NaN,3692,1449,1.25,30.23,0.60,NV099,AdUp07,Hu,BT46,...,NaN,NaN,NaN,NaN,NaN,CTL,1,CD4,PBT_batch5_library4_6donors_Gex,PBT_34


In [28]:
cd4_annotation.columns

Index(['clonotype.tag', 'nCount_RNA', 'nFeature_RNA', 'percent.mt',
       'percent.ribo', 'percent.hs', 'Run', 'Project', 'Species', 'donor',
       'tissue', 'site', 'Tumor.grade', 'Diagnosis', 'Diagnosis.subclass',
       'UMAP_1', 'UMAP_2', 'TRA.tag', 'TRB.tag', 'TRA.nt.chains.tag',
       'TRB.nt.chains.tag', 'TRA.aa.chains.tag', 'TRB.aa.chains.tag',
       'clon.size.tag', 'clon.proportion.tag', 'expDegree', 'sGroup_tag',
       'pattern', 'n_sG', 'tra.v', 'tra.j', 'trb.v', 'trb.j',
       'cell_classification_deconv', 'paper_cluster', 'Cell.Type',
       'library_name', 'donor_publicationID'],
      dtype='object')

In [29]:
cd4_annotation['donor_publicationID'].unique()

array(['PBT_12', 'PBT_01', 'PBT_02', 'PBT_10', nan, 'PBT_05', 'PBT_11',
       'PBT_35', 'PBT_25', 'PBT_26', 'PBT_24', 'PBT_20', 'PBT_32',
       'PBT_30', 'PBT_09', 'PBT_21', 'PBT_23', 'PBT_13', 'PBT_22',
       'PBT_29', 'PBT_31', 'PBT_04', 'PBT_14', 'PBT_07', 'PBT_17',
       'PBT_03', 'PBT_18', 'PBT_33', 'PBT_27', 'PBT_28', 'PBT_19',
       'PBT_08', 'PBT_38', 'PBT_34', 'PBT_06', 'PBT_36', 'PBT_15'],
      dtype=object)

In [30]:
cd4_subset = cd4_annotation[cd4_annotation['donor_publicationID'].isna()]
cd4_subset

,clonotype.tag,nCount_RNA,nFeature_RNA,percent.mt,percent.ribo,percent.hs,Run,Project,Species,donor,...,n_sG,tra.v,tra.j,trb.v,trb.j,cell_classification_deconv,paper_cluster,Cell.Type,library_name,donor_publicationID
cell_name,,,,,,,,,,,,,,,,,,,,,
AAACGGGGTCCGAGTC-1,clonotype311,3565,1299,2.97,35.75,0.11,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,TRBV28,TRBJ1-1,TCM,7,CD4,PBT_batch1_library1_7donors_Gex,NaN
AAACGGGTCACAACGT-1,clonotype310,5243,1843,1.85,29.66,1.18,NV035,AdUp01,Hu,NaN,...,1.0,TRAV16,TRAJ48,TRBV6-5,TRBJ2-1,TCM,7,CD4,PBT_batch1_library1_7donors_Gex,NaN
AAAGATGCAGACAAAT-1,NaN,3118,1336,2.76,29.31,0.13,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,TN,3,CD4,PBT_batch1_library1_7donors_Gex,NaN
AAAGATGGTGACAAAT-1,NaN,2457,1639,8.38,2.40,0.45,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,CTL,2,CD4,PBT_batch1_library1_7donors_Gex,NaN
AAAGCAAGTTTAGCTG-1,clonotype3435,3078,1528,3.18,18.99,0.32,NV035,AdUp01,Hu,NaN,...,NaN,TRAV3,TRAJ13,TRBV7-3,TRBJ2-5,TCM,0,CD4,PBT_batch1_library1_7donors_Gex,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCCTCCTGTAGCTAAA-12,clonotype46548,3707,1469,2.37,34.80,0.49,NV099,AdUp07,Hu,NaN,...,NaN,TRAV13-1,TRAJ3,TRBV12-3,TRBJ1-5,TCM,0,CD4,PBT_batch5_library4_6donors_Gex,NaN
GACCTGGGTTGTCTTT-12,NaN,798,611,0.72,6.21,0.12,NV099,AdUp07,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,Cell_Cycle,10,CD4,PBT_batch5_library4_6donors_Gex,NaN
GCATGTATCGGCCGAT-12,NaN,2650,986,3.70,41.85,0.15,NV099,AdUp07,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,TCM,0,CD4,PBT_batch5_library4_6donors_Gex,NaN


<span style="color:red">**PROBLEM:**</span> 898 cells from the CD4 file are from unknown donors

In [32]:
# inspect the CD8 data

cd8_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE221776/GSE221776_sc_PBT_CD8_umi.txt.gz'
cd8_input = pd.read_csv(cd4_path, sep='\t', index_col=0) # the first column contains gene names and is the index

print(cd8_input.head()) 
print(cd8_input.shape) # (15095, 15893)
# inspect the annotation file
cd4_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE221776/GSE221776_sc_PBT_CD4_annotation.txt.gz'
cd4_annotation = pd.read_csv(annotation_path, sep='\t', index_col=0) # the first column contains gene names and is the index

cd4_annotation

       gene_name  AAACCTGAGAGTACCG.1  AAACCTGAGCAATATG.1  AAACCTGAGCACACAG.1  \
1     AL627309.1                   0                   0                   0   
2     AP006222.2                   0                   0                   0   
3  RP11-206L10.3                   0                   0                   0   
4  RP11-206L10.2                   0                   0                   0   
5  RP11-206L10.9                   0                   0                   0   

   AAACCTGAGTGTCCAT.1  AAACCTGAGTTAAGTG.1  AAACCTGCACGGTAAG.1  \
1                   0                   0                   0   
2                   0                   0                   0   
3                   0                   0                   0   
4                   0                   0                   0   
5                   0                   0                   0   

   AAACCTGCAGCCTTTC.1  AAACCTGGTCCATCCT.1  AAACCTGGTTCTGAAC.1  ...  \
1                   0                   0                 

In [34]:
# inspect the annotation file
cd8_annotation_path = '/scratch/user/s4543064/xiaohan-john-project/data/GSE221776/GSE221776_sc_PBT_CD8_annotation.txt.gz'
cd8_annotation = pd.read_csv(cd8_annotation_path, sep='\t', index_col=0) # the first column contains gene names and is the index

cd8_annotation

,clonotype.tag,nCount_RNA,nFeature_RNA,percent.mt,percent.ribo,percent.hs,Run,Project,Species,donor,...,n_sG,tra.v,tra.j,trb.v,trb.j,cell_classification_deconv,paper_cluster,Cell.Type,library_name,donor_publicationID
cell_name,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGATATA-1,NaN,2131,1183,3.89,19.33,0.23,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,GZMK_HI,0,CD8,PBT_batch1_library1_7donors_Gex,NaN
AAACCTGCACCGAAAG-1,NaN,2275,1153,5.27,21.53,0.09,NV035,AdUp01,Hu,BT4,...,NaN,NaN,NaN,NaN,NaN,TRM,1,CD8,PBT_batch1_library1_7donors_Gex,PBT_10
AAACCTGGTCACCCAG-1,clonotype312,3297,1519,1.70,23.38,1.27,NV035,AdUp01,Hu,BT3,...,NaN,TRAV3,TRAJ34,NaN,NaN,GZMK_HI,0,CD8,PBT_batch1_library1_7donors_Gex,PBT_05
AAACCTGTCCTCTAGC-1,NaN,1263,843,5.06,9.72,0.00,NV035,AdUp01,Hu,BT3,...,NaN,NaN,NaN,NaN,NaN,TRM,1,CD8,PBT_batch1_library1_7donors_Gex,PBT_05
AAACGGGAGAACTCGG-1,clonotype247,1529,916,5.88,16.07,0.07,NV035,AdUp01,Hu,BT3,...,NaN,NaN,NaN,TRBV19,TRBJ2-1,TRM,1,CD8,PBT_batch1_library1_7donors_Gex,PBT_05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCACAAACC-10,clonotype45287,1560,1009,0.83,10.71,0.13,NV099,AdUp07,Hu,BT38,...,NaN,TRAV19,TRAJ24,TRBV5-4,TRBJ2-1,GZMK_HI,0,CD8,PBT_batch5_library2_6donors_Gex,PBT_15
TTTGGTTTCCGAGCCA-10,NaN,3079,1385,1.75,21.20,3.67,NV099,AdUp07,Hu,BT38,...,NaN,NaN,NaN,NaN,NaN,CD16p_Effector,5,CD8,PBT_batch5_library2_6donors_Gex,PBT_15
TTTGGTTTCGAGAACG-10,NaN,4367,1671,1.60,25.39,0.16,NV099,AdUp07,Hu,BT38,...,NaN,NaN,NaN,NaN,NaN,GZMK_HI,0,CD8,PBT_batch5_library2_6donors_Gex,PBT_15


In [35]:
cd8_annotation.columns

Index(['clonotype.tag', 'nCount_RNA', 'nFeature_RNA', 'percent.mt',
       'percent.ribo', 'percent.hs', 'Run', 'Project', 'Species', 'donor',
       'tissue', 'site', 'Tumor.grade', 'Diagnosis', 'Diagnosis.subclass',
       'UMAP_1', 'UMAP_2', 'TRA.tag', 'TRB.tag', 'TRA.nt.chains.tag',
       'TRB.nt.chains.tag', 'TRA.aa.chains.tag', 'TRB.aa.chains.tag',
       'clon.size.tag', 'clon.proportion.tag', 'expDegree', 'sGroup_tag',
       'pattern', 'n_sG', 'tra.v', 'tra.j', 'trb.v', 'trb.j',
       'cell_classification_deconv', 'paper_cluster', 'Cell.Type',
       'library_name', 'donor_publicationID'],
      dtype='object')

In [36]:
cd8_annotation['donor_publicationID'].unique()

array([nan, 'PBT_10', 'PBT_05', 'PBT_12', 'PBT_01', 'PBT_35', 'PBT_11',
       'PBT_02', 'PBT_26', 'PBT_25', 'PBT_32', 'PBT_24', 'PBT_30',
       'PBT_09', 'PBT_20', 'PBT_21', 'PBT_29', 'PBT_04', 'PBT_07',
       'PBT_13', 'PBT_22', 'PBT_14', 'PBT_23', 'PBT_17', 'PBT_31',
       'PBT_03', 'PBT_19', 'PBT_33', 'PBT_18', 'PBT_27', 'PBT_28',
       'PBT_08', 'PBT_38', 'PBT_15', 'PBT_06', 'PBT_34', 'PBT_37',
       'PBT_36', 'PBT_16'], dtype=object)

In [37]:
len(cd8_annotation['donor_publicationID'].unique())

39

In [38]:
cd8_subset = cd8_annotation[cd8_annotation['donor_publicationID'].isna()]
cd8_subset

,clonotype.tag,nCount_RNA,nFeature_RNA,percent.mt,percent.ribo,percent.hs,Run,Project,Species,donor,...,n_sG,tra.v,tra.j,trb.v,trb.j,cell_classification_deconv,paper_cluster,Cell.Type,library_name,donor_publicationID
cell_name,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCGATATA-1,NaN,2131,1183,3.89,19.33,0.23,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,GZMK_HI,0,CD8,PBT_batch1_library1_7donors_Gex,NaN
AAAGATGAGACTGGGT-1,clonotype109,1307,837,1.07,18.26,0.46,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,TRBV7-6,TRBJ1-6,GZMK_HI,0,CD8,PBT_batch1_library1_7donors_Gex,NaN
AAATGCCAGACTGGGT-1,clonotype42,4214,1918,1.16,24.56,0.19,NV035,AdUp01,Hu,NaN,...,1.0,TRAV39,TRAJ32,TRBV20-1,TRBJ1-5,MAIT,6,CD8,PBT_batch1_library1_7donors_Gex,NaN
AAATGCCTCACATACG-1,NaN,1832,1036,4.04,19.65,0.00,NV035,AdUp01,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,TCF7high_CCR7high,2,CD8,PBT_batch1_library1_7donors_Gex,NaN
AACACGTCAATGTAAG-1,clonotype3918,2215,1233,2.93,8.88,0.18,NV035,AdUp01,Hu,NaN,...,NaN,TRAV13-1,TRAJ49,TRBV10-3,TRBJ2-3,GZMK_HI,0,CD8,PBT_batch1_library1_7donors_Gex,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCAGCTCAGTATCGAA-10,clonotype44132,1430,770,3.98,22.29,1.26,NV099,AdUp07,Hu,NaN,...,NaN,NaN,NaN,TRBV25-1,TRBJ1-2,TCF7high_CCR7low,3,CD8,PBT_batch5_library2_6donors_Gex,NaN
TCAGGATTCCACGCAG-10,NaN,3231,1401,2.23,29.56,0.19,NV099,AdUp07,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,MAIT,4,CD8,PBT_batch5_library2_6donors_Gex,NaN
TCCACACCATGGGAAC-10,NaN,2147,988,3.45,31.94,0.28,NV099,AdUp07,Hu,NaN,...,NaN,NaN,NaN,NaN,NaN,MAIT,4,CD8,PBT_batch5_library2_6donors_Gex,NaN


<span style="color:red">**PROBLEM:**</span> 1569 cells from the CD8 file are from unknown donors

### 2. Overall AnnData object of the dataset

<span style="color:red">**IMPORTANT:**</span> transpose the DataFrame.values to match the AnnData.X

1. `DataFrame.columns`: cell barcodes, which go into `.obs`
2. `DataFrame.index`: gene names, `.var`
3. `DataFrame.values`: the transpose of the expression matrix, `.X`

In [10]:
matrix = scipy.sparse.csr_matrix(input.values.T)
obs_name = pd.DataFrame(index=input.columns)
var_name = pd.DataFrame(input.index)
var_name.rename(columns={'Gene': 'gene_symbols'}, inplace=True)

sample = anndata.AnnData(X=matrix, obs=obs_name, var=var_name)
print(sample)

# Create an observation metric info to store related features
obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
obs_metrics['cancer_type'] = 'H3K27M-glioma'
obs_metrics['dataset'] = 'GSE102130'
obs_metrics['tissue'] = 'brain'
obs_metrics['sample_barcode'] = 'GSE102130_K27Mproject.RSEM.vh20170621'
obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)

sample.obs = obs_metrics
sample.obs.set_index("uni_barcode", drop=False, inplace=True)
print(sample)

# save the anndata object
sample.write_h5ad('/scratch/user/s4543064/xiaohan-john-project/write/GSE102130/GSE102130_K27Mproject.RSEM.vh20170621_uni.h5ad', compression="gzip")

/home/s4543064/miniforge3/envs/test/lib/python3.12/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 4058 × 23686
    var: 'gene_symbols'
AnnData object with n_obs × n_vars = 4058 × 23686
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_symbols'


In [11]:
sample.obs

,cancer_type,dataset,tissue,uni_barcode
uni_barcode,,,,
GSE102130_MUV1-P04-B12,H3K27M-glioma,GSE102130,brain,GSE102130_MUV1-P04-B12
GSE102130_MUV1-P04-C08,H3K27M-glioma,GSE102130,brain,GSE102130_MUV1-P04-C08
GSE102130_MUV1-P04-D09,H3K27M-glioma,GSE102130,brain,GSE102130_MUV1-P04-D09
GSE102130_MUV1-P04-D10,H3K27M-glioma,GSE102130,brain,GSE102130_MUV1-P04-D10
GSE102130_MUV1-P04-E03,H3K27M-glioma,GSE102130,brain,GSE102130_MUV1-P04-E03
...,...,...,...,...
GSE102130_Oligo-P22-H03,H3K27M-glioma,GSE102130,brain,GSE102130_Oligo-P22-H03
GSE102130_Oligo-P22-H05,H3K27M-glioma,GSE102130,brain,GSE102130_Oligo-P22-H05
GSE102130_Oligo-P22-H06,H3K27M-glioma,GSE102130,brain,GSE102130_Oligo-P22-H06


### 3. Confirmation of created AnnData object

In [13]:
output = '/scratch/user/s4543064/xiaohan-john-project/write/GSE102130/GSE102130_K27Mproject.RSEM.vh20170621_uni.h5ad'
sample = anndata.read_h5ad(output)
print(sample)

AnnData object with n_obs × n_vars = 4058 × 23686
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_symbols'


### 4. Convert AnnData objects to SingleCellExperiment objects

In [14]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE102130')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))